# MusicXML Exploration with Music21
The following code attempts to understand and make notes about how .mxl and .xml files work with Music21.

In [2]:
# imports and constants
import music21
from pprint import pprint
from difficulty import *

In [4]:
# score = music21.converter.parse("../music/Beethoven_Symphony_9__Op._125.mxl")

# This used to break, reporting the error:
# MusicXMLImportException: In part (Flauto piccolo), measure (959): found unknown MusicXML type: None
# Then I opened the file in MuseScore, had it ignore the issue, and exported it back out to .mxl, and overwrote the original
# Now, it works but takes FOREVER (generally more than 4 min)
# Additionally, it seems like 
score = music21.converter.parse("../music/xml/Beethoven_Symphony_9__Op._125.mxl")

In [10]:
def is_bassoon_name(candidate):
    bsn_names = ["bassoon", "fagotti", "basson", "fagott", "fagotto", "fagot", "fagote", "dulcian"]
    for name in bsn_names:
        if candidate == name:
            return True
    return False

In [9]:
# find bassoon part in music21 score
bsn_part = None
for part in score.parts:
    print(part.partName.lower())
    if part.partName.lower() == 'bassoon':
        print(part.partName)
        bsn_part = part
        break

if bsn_part == None:
    raise Exception("Bassoon part not found")

flauto piccolo
flauto 1
flauto 2
oboe 1
oboe 2
clarinetto 1 in c
clarinetto 1 in b
clarinetto 1 in a
clarinetto 2 in c
clarinetto 2 in b
clarinetto 2 in a
fagotto 1
fagotto 2
controfagotto
corno 1 in d
corno 1 in b
corno 2 in d
corno 2 in b
corno 3 in e♭
corno 3 in d
corno 3 in b basso
corno 4 in e♭
corno 4 in d
corno 4 in b basso
tromba 1 in d
tromba 1 in b
tromba 2 in d
tromba 2 in b
trombone alto
trombone tenore
trombone basso
timpani in d, a
timpani in f, f basso
timpani in f, b
triangolo
cinelli
gran tamburo
soprano solo
alto solo
tenore solo
basso solo
soprano
alto
tenore
tenore 1
tenore 2
basso
violini i
violini ii
viole
viole
violoncelli
contrabbassi


Exception: Bassoon part not found

In [6]:
bsn_part.show('musicXML')

In [7]:
bassoon = score[6]  # parts count from 0-16; 3 is bassoon
print(bassoon.partName.lower())
print(type(bassoon))
excerpt = bassoon.measures(0, 30)
# excerpt.show()
for note in excerpt.flat:
    print(note.next)

bassoon
<class 'music21.stream.base.Part'>
<bound method Music21Object.next of <music21.instrument.Bassoon 'P4: Bassoon: Bassoon'>>
<bound method Music21Object.next of <music21.layout.SystemLayout>>
<bound method Music21Object.next of <music21.layout.StaffLayout distance 121.21, staffNumber 1, staffSize None, staffLines None>>
<bound method Music21Object.next of <music21.clef.BassClef>>
<bound method Music21Object.next of <music21.key.KeySignature of 3 flats>>
<bound method Music21Object.next of <music21.meter.TimeSignature 2/4>>
<bound method Music21Object.next of <music21.note.Rest half>>
<bound method Music21Object.next of <music21.dynamics.Crescendo <music21.note.Note B-><music21.note.Note B->>>
<bound method Music21Object.next of <music21.note.Rest half>>
<bound method Music21Object.next of <music21.note.Rest half>>
<bound method Music21Object.next of <music21.note.Rest half>>
<bound method Music21Object.next of <music21.note.Rest half>>
<bound method Music21Object.next of <music2

In [4]:
# adds a note to a dict if not in it, increments if is in it
def note_record(note, notes):
    if note.nameWithOctave in notes:
        notes[note.nameWithOctave] += 1
        # notes[note.pitch] += 1
    else:
        notes[note.nameWithOctave] = 1
        # notes[note.pitch] = 1

# def note_record(note):
#     if note.pitch in notes_dict:
#         notes_dict[note.pitch] += 1
#     else:
#         notes_dict[note.pitch] = 1
        # if note.nameWithOctave == 'F#2':
        #     pprint(note.__dict__)

def inc_accidental(note, num):
    # check if it's in the key
    if note.pitch.accidental is not None:
        return num + 1
    return num


In [5]:
# Go through score and record the number of occurrences of each note
notes_dict_bassoon_1 = {}
note_pairs_dict_b1 = {}
notes_dict_bassoon_2 = {}
note_pairs_dict_b2 = {}
num_accidentals = 0
# fsharp = music21.note.Note('F#2')

for el in bsn_part.recurse().notes:
    if type(el)== music21.chord.Chord:
        sortedNotes = el.sortAscending()
        note_record(sortedNotes[0], notes_dict_bassoon_2)
        note_record(sortedNotes[1], notes_dict_bassoon_1)
        # for note in el.notes:
        #     note_record(note)
        #     num_accidentals = inc_accidental(note, num_accidentals)
                
    else:
        note_record(el, notes_dict_bassoon_2)
        note_record(el, notes_dict_bassoon_1)
        # note_record(el)
        # num_accidentals = inc_accidental(el, num_accidentals)

print("Number of accidentals:", num_accidentals) # I think this is wrong
print("-------------- Bassoon 1 --------------")
pprint(notes_dict_bassoon_1)
print("-------------- Bassoon 2 --------------")
pprint(notes_dict_bassoon_2)

Number of accidentals: 0
-------------- Bassoon 1 --------------
{'A#3': 5,
 'A-2': 1,
 'A-3': 1,
 'A2': 22,
 'A3': 227,
 'B-2': 5,
 'B-3': 15,
 'B2': 8,
 'B3': 68,
 'C#3': 15,
 'C#4': 22,
 'C3': 11,
 'C4': 31,
 'D#3': 2,
 'D#4': 3,
 'D3': 61,
 'D4': 70,
 'E-3': 3,
 'E-4': 9,
 'E3': 47,
 'E4': 30,
 'F#3': 52,
 'F3': 26,
 'F4': 10,
 'G#3': 10,
 'G2': 1,
 'G3': 91}
-------------- Bassoon 2 --------------
{'A#3': 5,
 'A-2': 1,
 'A-3': 1,
 'A2': 22,
 'A3': 227,
 'B-2': 5,
 'B-3': 15,
 'B2': 8,
 'B3': 68,
 'C#3': 15,
 'C#4': 22,
 'C3': 11,
 'C4': 31,
 'D#3': 2,
 'D#4': 3,
 'D3': 61,
 'D4': 70,
 'E-3': 3,
 'E-4': 9,
 'E3': 47,
 'E4': 30,
 'F#3': 52,
 'F3': 26,
 'F4': 10,
 'G#3': 10,
 'G2': 1,
 'G3': 91}
